In [ ]:
#Начинаем, загружаем датафрейм из таблицы, чистим неправильные ответы
import pandas as pd 
df=pd.read_excel('/content/test.xlsx')# читаем таблицу
df_copy=df.copy#делаем копию на всякий случай
df = df.drop(df[df.status =="wrong"].index) #выкидываем те, которые имеют статус "wrong"
df

,submission_id,step_id,user_id,last_name,first_name,attempt_time,submission_time,status
4,223729406,1356110,51265829,Фульман,Виталий,1591517128,1591517177,correct
6,223764048,1357401,51265829,Фульман,Виталий,1591522209,1591522310,correct
7,223780375,1356112,51265829,Фульман,Виталий,1591524540,1591524568,correct
8,223793071,1356112,246695214,Movavi,Школа,1591526195,1591526221,correct
9,223793513,1356110,246695214,Movavi,Школа,1591526241,1591526274,correct
...,...,...,...,...,...,...,...,...
289,226503901,1356110,252331072,Ведерников,Андрей,1591878396,1591878535,correct
290,226504368,1356110,252335096,Ведерников,Александр,1591873787,1591878575,correct
292,226509318,1356112,252331072,Ведерников,Андрей,1591878545,1591878998,correct
293,226537111,1357401,252331072,Ведерников,Андрей,1591879007,1591881469,correct


In [ ]:
#Делаем чистый датафрейм с теми-же колонками, пишем в него только первую успешную попытку по каждому степу каждого пользователя.
df_clear=df.copy() #копируем основной датафрейм
df_clear=df_clear.drop(df.index) #оставляем там только колонки
while len(df)!=0: #на всякий случай по количеству элементов циркулируем
  first_uid=df.user_id.iloc[0] #берём из первого user_id
  first_si=df.step_id.iloc[0] #берём из первого step_id
  filter1=df["user_id"]==first_uid
  filter2=df["step_id"]==first_si
  df_qe=df.loc[filter1&filter2] #выборка с сопадающими user_id и step_id как у первого элемента
  df_qe.sort_values("submission_time")# сортируем выборку по возрастанию submission_time
  df_temp=df_qe.iloc[[0]] #берем первого
  df_clear=pd.concat([df_temp,df_clear]) #добавляем в новый датафрейм
  df=df.loc[~(filter1&filter2)]#убираем из основной выборки всех с сопадающими user_id и step_id как у первого элемента
df_clear
  

,submission_id,step_id,user_id,last_name,first_name,attempt_time,submission_time,status
293,226537111,1357401,252331072,Ведерников,Андрей,1591879007,1591881469,correct
292,226509318,1356112,252331072,Ведерников,Андрей,1591878545,1591878998,correct
290,226504368,1356110,252335096,Ведерников,Александр,1591873787,1591878575,correct
289,226503901,1356110,252331072,Ведерников,Андрей,1591878396,1591878535,correct
281,226494578,1357401,252331275,Вадим,Колесников,1591876628,1591877704,correct
...,...,...,...,...,...,...,...,...
9,223793513,1356110,246695214,Movavi,Школа,1591526241,1591526274,correct
8,223793071,1356112,246695214,Movavi,Школа,1591526195,1591526221,correct
7,223780375,1356112,51265829,Фульман,Виталий,1591524540,1591524568,correct
6,223764048,1357401,51265829,Фульман,Виталий,1591522209,1591522310,correct


In [ ]:
#Создаем датафрейм для финальной статистики, пишем туда одну запись по пользователю с подсчетом количества правильных ответов и последним submission_time
df_final=df_clear.copy() #финальная статистика
df_final=df_final.drop(df_final.index) # удаляем все содержимое
while len(df_clear)!=0:# идём по всему датафрейму
  df_temp=df_clear.loc[df_clear["user_id"]==df_clear.user_id.iloc[0]]#выборка всех пользователей с u_id как у первого
  df_clear.step_id.iloc[0]=len(df_temp) #Пишем в step_id сколько таких
  df_clear.submission_time.iloc[0]=df_clear.submission_time.iloc[len(df_temp)-1] #Пишем в submission_time первой записи submission_time последного такого с u_id
  df_final=pd.concat([df_final,df_clear.iloc[[0]]])#пишем первого в финальный датафрейм
  df_clear=df_clear.drop(df_temp.index) #удаляем всех у кого u_id как у первого
df_final.rename(columns={'step_id': 'points'}, inplace=True)#исправляем step_id на Points
df_final

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


,submission_id,points,user_id,last_name,first_name,attempt_time,submission_time,status
293,226537111,3,252331072,Ведерников,Андрей,1591879007,1591878575,correct
290,226504368,1,252335096,Ведерников,Александр,1591873787,1591878575,correct
281,226494578,3,252331275,Вадим,Колесников,1591876628,1591877449,correct
277,226492387,3,252331109,Георгий,Новиков,1591874748,1591877086,correct
276,226491702,3,252336642,Логинов,Владимир,1591876165,1591876939,correct
271,226487627,3,252331878,Печуркин,Матвей,1591876703,1591876690,correct
266,226486042,3,252330698,Никонов,Максим,1591876676,1591875475,correct
240,226459789,3,252331628,Шалыгин,Игорь,1591874316,1591874248,correct
231,225870744,3,251717664,Быков,Андрей,1591793278,1591793014,correct
223,225814587,2,251664379,Кайда,Никита,1591785822,1591789830,correct


In [ ]:
#итоговый рейтинг, пишем в таблицу и выводим

df_rating=df_final.sort_values(["points","submission_time"],ascending=[False,True])
df_rating.to_excel("rating_all.xlsx")
df_rating

,submission_id,points,user_id,last_name,first_name,attempt_time,submission_time,status
7,223780375,3,51265829,Фульман,Виталий,1591524540,1591517177,correct
10,223794014,3,246695214,Movavi,Школа,1591526229,1591526221,correct
55,224415267,3,250365834,Андрюшечко,Данил,1591614234,1591614130,correct
70,224421502,3,136523573,Копалкин,Иван,1591614386,1591614379,correct
78,224427742,3,250366178,Калачиков,Фёдор,1591615125,1591615115,correct
79,224429295,3,250366319,Сазонов,Евгений,1591614573,1591615620,correct
80,224430137,3,250365895,АБросИмоВ,ИЛья,1591615089,1591616229,correct
81,224432756,3,250366026,Белоусов,Егор,1591615350,1591616377,correct
113,225030159,3,15455235,Оконечников,Павел,1591692936,1591693640,correct
155,225163330,3,251032146,Красноперов,Константин,1591702514,1591702888,correct


In [ ]:
#берём только первых трёх из итогового рейтинга, пишем в таблицу и выводим.
df_top=df_rating.head(3)
df_top.to_excel("winner_olympiad.xlsx")
df_top

NameError: ignored